\title{Prémiers pas avec LAMMPS}
\author{YOUBI Marie Rose,rose.youbi@facsciences-uy1.cm }
\maketitle

## Définition et déscription

LAMMPS est un code de dynamique moléculaire (MD) classique qui modélise des ensembles de particules à l'état liquide, solide ou gazeux. Il utilise une variété de potentiels interatomiques (champs de force) et de conditions aux limites pour modéliser des systèmes macroscopiques tel que **les polymères, les molécules biologiques, solides (métaux, céramiques, oxydes) etc.** en 2d ou 3d.\
Écrit en C++ et nécessite un compilateur qui est au moins compatible avec la norme C++-11, il peut être construit et exécuté sur importe quel type d'ordinateur mais sera plus efficace sur machines parallèles qui supporte la bibliothèque de passage de messages MPI. 



## Installation  sur linux
### Téléchargement

Les différents étapes pour télcharger LAMMPS sont :
1. Télecharger Lammps sur le site web, https://www.lammps.org/download.html ;
1. Choisir la version la plus récente et stable;
1. Cliquer sur le bouton "download now" . 


### Installation 

Pour installer, je vous invite à suivre les étapes données dans la vidéo suivante : https://www.youtube.com/watch?v=3Tb8SsO0Dzg&t=87s

**NB :** Dans cette vidéo l'excecutable intaller est **mpi** mais nous utilisons **serial**. Ne vous inquiéter pas le processus est le même. 

## Executer LAMMPS dans jupyter notebook
### Télécharger le fichier input

Pour télecharger un fichier input, on ajoute la commande `%%writefile nom du script.in` au début du notebook. Ainsi, le contenu sera sauvegarder dans un fichier `.in` .

In [3]:
%%writefile calc_fcc.in
######################################
# LAMMPS INPUT SCRIPT
# Find minimum energy fcc (face-centered cubic) atomic configuration
# Mark Tschopp
# Syntax, lmp_exe < calc_fcc.in

######################################
# INITIALIZATION
clear 
units metal 
dimension 3 
boundary p p p 
atom_style atomic 
atom_modify map array

######################################
# ATOM DEFINITION
lattice fcc 4 orient x 1 1 0 orient y -1 1 0 orient z 0 0 1  
region box block 0 1 0 1 0 1 units lattice
create_box 1 box
create_atoms 1 box
replicate 1 1 1

######################################
# DEFINE INTERATOMIC POTENTIAL
pair_style eam/alloy 
pair_coeff * * Al99.eam.alloy Al
neighbor 2.0 bin 
neigh_modify delay 10 check yes 
 
######################################
# DEFINE COMPUTES 
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 

#####################################################
# MINIMIZATION
reset_timestep 0 
fix 1 all box/relax iso 0.0 vmax 0.001
thermo 10 
thermo_style custom step pe lx ly lz press c_eatoms 
min_style cg 
minimize 1e-25 1e-25 5000 10000 

variable natoms equal "count(all)" 
variable teng equal "c_eatoms"
variable length equal "lx"
variable ecoh equal "v_teng/v_natoms"

print "Total energy (eV) = ${teng};"
print "Number of atoms = ${natoms};"
print "Lattice constant (Angstoms) = ${length};"
print "Cohesive energy (eV) = ${ecoh};"

print "All done!"

Overwriting calc_fcc.in


On peut également utiliser Python pour ecrire le script en ouvrant un fichier et y ajouter tout le contenu neccessaire.

In [4]:
file = open("calc_fcc.in", "w")
file.write("clear\n ")
file.write("units metal \n ")
file.write("dimension 3 \n ")
file.write("boundary p p p \n ")
file.write("atom_style atomic \n ")
file.write("atom_modify map array \n ") 
file.write("lattice fcc 4 \n ")
file.write("region box block 0 1 0 1 0 1 units lattice \n ")
file.write("create_box 1 box \n ")
file.write("lattice fcc 4 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1\n ")
file.write("create_atoms 1 box \n ")
file.write("replicate 1 1 1\n ")
file.write("pair_style eam/alloy\n ") 
file.write("pair_coeff * * Al99.eam.alloy Al\n ")
file.write("neighbor 2.0 bin \n ")
file.write("neigh_modify delay 10 check yes\n ")
file.write("compute eng all pe/atom \n ")
file.write("compute eatoms all reduce sum c_eng \n ")
file.write("reset_timestep 0 \n ")
file.write("fix 1 all box/relax iso 0.0 vmax 0.001\n ")
file.write("thermo 10 \n ")
file.write("thermo_style custom step pe lx ly lz press c_eatoms \n ")
file.write("min_style cg \n ")
file.write("minimize 1e-25 1e-25 5000 10000 \n ")
file.write("variable natoms equal \"count(all)\"\n ") 
file.write("variable teng equal \"c_eatoms\"\n ")
file.write("variable length equal \"lx\"\n ")
file.write("variable ecoh equal \"v_teng/v_natoms\"\n ")
file.write("print \"Total energy (eV) = ${teng};\"\n ")
file.write("print \"Number of atoms = ${natoms};\"\n ")
file.write("print \"Lattice constant (Angstoms) = ${length};\"\n ")
file.write("print \"Cohesive energy (eV) = ${ecoh};\"\n ")
file.write("print \"All done!\"\n ")
file.close()

### Comprendre les differentes parties du script

Le `#` permet de mettre en comentaire. c'est à dire que tout ce qui est écrit après le `#` n'est pas prise en compte lors de l'execution de fichier.

1. La première partie nommée **initialize Simulation** définie comme suite :
<br>
<div class="alert alert-block alert-info">
# ---------- Initialize Simulation ---------------------  <br>
clear  <br>
units metal  <br>
dimension 3  <br>
boundary p p p  <br>
atom_style atomic  <br>
atom_modify map array <br>
</div>

Initialise les simulations. La commande `clear` efface toute la mémoire. La commande `units` spécifie les unités qui seront utilisées pour le reste de la simulation par exemple `metal` utilise Angströms et eV comme unité de base (on peut aussi utiliser `lj`). La commande `dimension 3` que la simulation sera fait en 3D (on peut également utiliser le 2D ou le 1D). Le `boundary p p p` spécifie que notre système sera périodiques dans les directions x, y et z (on peut aussi utiliser `s s s` pour les systèmes non pèriodiques).

2. La deuxième partie nommée **Création des atomes** définie par : 
<br>
<div class="alert alert-block alert-info">
# ---------- Create Atoms ---------------------  <br>
lattice 	fcc 4 <br>
region	box block 0 1 0 1 0 1 units lattice <br>
create_box	1 box <br> <br>
    
lattice	fcc 4 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1   <br>
create_atoms 1 box <br>
replicate 1 1 1 <br></div>

Permet de définir les paramèttres des atomes à utiliser.

La commande `lattice` spécifie quel type de réseau est utilisé (**fcc, bcc, hcp**, etc.) et le nombre qui suit spécifie la constante du réseau. La commande `region` spécifie la boite de simulation et ses dimensions. Ici, nous avons utilisé des unités de réseau (*units lattice*) et spécifié que la boîte de simulation doit être d'une unité de réseau dans chaque direction. La commande `create_box` qui suit utilisera les paramètres indiqués dans la commande `region` pour créer réellement la boîte. Le chiffre placer devant specifie le nombre de boite créer.  La commande `replicate` peut être utilisée pour répliquer la cellule périodique dans chaque direction.

3. La troisième partie nommée **Definir le potentiel interatomique** définie par :
<br>
<div class="alert alert-block alert-info">
# ---------- Define Interatomic Potential --------------------- <br>
pair_style eam/alloy  <br>
pair_coeff * * Al99.eam.alloy Al <br>
neighbor 2.0 bin  <br>
neigh_modify delay 10 check yes  <br>
</div>

La commande `pair_style` spécifie le type de potentiel interatomique qui sera utilisé. La commande `pair_coeff` spécifie le fichier dans lequel les coefficients de potentiel de paire sont stockés. L'extension de fichier pour le fichier de potentiel interatomique peut parfois donner un indice sur le format utilisé.

Exemple: __eam.alloy = eam/alliage__.

4. La quatrième partie nommée **Définir les paramètres** définie par :
<br>
<br>
<div class="alert alert-block alert-info">
# ---------- Define Settings --------------------- <br>
compute eng all pe/atom <br>
compute eatoms all reduce sum c_eng <br>
</div>

Dans la première commande `compute`, une variable nommée `eng` est définie pour stocker l'énergie potentielle de chaque atome. Dans la deuxième commande `compute`, une variable nommée `eatoms` est définie pour stocker la somme de toutes les valeurs `eng`. C'est juste pour montrer comment utiliser les calculs avec des variables définies par l'utilisateur. 

**NB :** La colonne d'énergie `pe` pendant la minimisation est équivalente à la colonne `c_eatoms`, comme prévu.

5. La cinquième partie nommée **Execution de minimisation** définie par :
<br>
<div class="alert alert-block alert-info">
# ---------- Run Minimization --------------------- <br> 
reset_timestep 0  <br>
fix 1 all box/relax iso 0.0 vmax 0.001 <br>
thermo 10  <br>
thermo_style custom step pe lx ly lz press c_eatoms  <br>
min_style cg  <br>
minimize 1e-25 1e-25 5000 10000   <br>
</div>

%%Le reset_timestep fait exactement cela.
        La commande `fix` utilise le paramètre `box/relax`, dans lequel toutes les directions (`iso`) sont relâchées à une pression de `0,0 Pa` pour tous les atomes (`all`). La commande `thermo` spécifie la sortie pendant la minimisation. `thermo_style` spécifie quel type de sortie est affiché à l'écran. Ici, on utilise une liste de mésures `custom`, y compris le `step`, l'énergie potentielle (`pe`), la longueur de la boîte en x, y et z (`lx, ly, lz`, respectivement), la `press`ion et la variable calculée `eatoms`. Le `min_style` spécifie que le gradient conjugué (`cg`) sera utilisé pour la minimisation et le `minimize` démarre la minimisation. 
 %%*c'est-à-dire*, les limites des cellules de simulation sont assouplies de la constante de réseau spécifiée (4 Angströms) à la constante de réseau réelle (4,05 Angströms).

6. La sixième partie nommée **Execution de minimisation** définie par :
<br>
<div class="alert alert-block alert-info">
variable natoms equal "count(all)" <br>
variable teng equal "c_eatoms" <br>
variable length equal "lx" <br>
variable ecoh equal "v_teng/v_natoms" 
</div>

Cette section définit certaines variables, telles que le nombre d'atomes `natom`, `teng` pour l'énergie potentielle totale,`length` pour la longueur de la cellule de simulation et `ecoh` pour l'énergie de cohésion de Al (puisque c'est le potentiel de Al qu'on utilise).

<br>
<div class="alert alert-block alert-info">
print "Total energy (eV) = \${teng};" <br>
print "Number of atoms = \${natoms};" <br>
print "Lattice constant (Angstoms) = \${length};" <br>
print "Cohesive energy (eV) = \${ecoh};" <br>
print "All done!"
</div>

Cette section imprime ces valeurs à l'écran et dans le fichier journal. 
Le `${}` est utilisé pour insérer les variables définies précédemment.


### Télecharger le potentiel interatomique à tuliser

Pour télechareger le potentiel interatomique dont-on a besoin, on peux :
1.  Accédez à la page Web du potentiel interatomique du NIST, https://www.ctcms.nist.gov/potentials/
1. Cliquez sur l'atome dont le potentiel veux être télécharger (__Al__ pour un potentiel d'élément unique en aluminium)
1. Cliquez sur le potentiel de votre choix. Pour cet exemple, utilisez le potentiel Mishin*et al.*, Al99.eam.alloy
1. Enregistrez-le dans le même répertoire que l'exécutable LAMMPS. 
si vous êtes sur linux, vous pouvez utiliser la commande suivante pour télécharger `wget -c + lien du site`

**Exemple:** `wget -c https://www.ctcms.nist.gov/potentials/Download/1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Al/2/Al99.eam.alloy`

## Executer en utilisant LAMMPS dans jupyter notebook

L'execution du fichier input est fait à partir de la commande ` LAMMPS_executable < input(file)`.

Si le fichier input n'est pas dans le même dossier que l'executable, vous devriez définir le chemin vers l'executable afin de pouvoir executer le fichier 

In [5]:
!"/media/mr-youbi/MRLOGICIEL/lammps-3Mar2020/lammps-3Mar20/src/lmp_serial" < calc_fcc.in

LAMMPS (3 Mar 2020)
Lattice spacing in x,y,z = 4 4 4
Created orthogonal box = (0 0 0) to (4 4 4)
  1 by 1 by 1 MPI processor grid
Lattice spacing in x,y,z = 4 4 4
Created 4 atoms
  create_atoms CPU = 0.000406981 secs
Replicating atoms ...
  orthogonal box = (0 0 0) to (4 4 4)
  1 by 1 by 1 MPI processor grid
  4 atoms
  replicate CPU = 0.00079298 secs
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 8.28721
  ghost atom cutoff = 8.28721
  binsize = 4.1436, bins = 1 1 1
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair eam/alloy, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d/newton
      bin: standard
Setting up cg style minimization ...
  Unit style    : metal
  Current step  : 0
Per MPI rank memory allocation (min/avg/max) = 4.495 | 4.495 | 4.495 Mbytes
Step PotEng Lx Ly Lz Press c_eatoms 
       0   -1